In [ ]:
from pycil.utils.data_manager import DataManager
import torch
from torch.utils.data import DataLoader
import numpy as np
from pathlib import Path
from utils.inc_net import DERNet
from tqdm import tqdm

In [ ]:
args = {
    'dataset': 'LogoDet-3K_cropped',
    'memory_size': 2000,
    'memory_per_class': 1000,
    'fixed_memory': True,
    'shuffle': True,
    'init_cls': 30, 'increment': 10,
    'model_name': 'der',
    'data_augmentation': True,
    'seed': 830694,
    'dropout': 0.5,
    'convnet_type': 'resnet34',
    'pretrained': True,
    'adam': True,
    'onlytop': True,
    'baseline': False,
    'device': torch.device('cpu')
}

data_manager = DataManager(
    args['dataset'], args['shuffle'], args['seed'], args['init_cls'],
    args['increment'], args.get('data_augmentation', False)
)

In [ ]:
# Test split
test_dataset = data_manager.get_dataset(
    np.arange(0, 100),
    source='test', mode='test'
)
test_loader = DataLoader(
    test_dataset, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
model_cil = DERNet(args['convnet_type'], args['pretrained'], dropout=args.get('dropout'))

for task, n_update in enumerate(np.cumsum([30] + [10] * 7)):
    print(n_update)
    model_cil.update_fc(n_update)
    if task > 0:
        for i in range(task):
            for p in model_cil.convnets[i].parameters():
                p.requires_grad = False

base = Path('/Users/gianlucagiudice/Documents/github/logo-detection-recognition/weights')
model_cil.load_state_dict(torch.load(base / 'state_dict.pt', map_location=args['device'])['state_dict'])

In [ ]:
model_cil.eval()

y_pred, y_true = [], []
for _, (_, inputs, targets) in tqdm(enumerate(test_loader), total=len(test_loader)):
    inputs = inputs.to(args['device'])
    with torch.no_grad():
        outputs = model_cil(inputs)['logits']
    predicts = torch.topk(outputs, k=1, dim=1, largest=True, sorted=True)[1]  # [bs, topk]
    y_pred.append(predicts.cpu().numpy())
    y_true.append(targets.cpu().numpy())

y_pred = np.concatenate(y_pred).flatten()
y_true = np.concatenate(y_true).flatten()

In [ ]:
top1_acc = ((y_pred.flatten() == y_true).sum() / len(y_true)) * 100
print(f'Top-1 accuracy: {top1_acc:.3f}')